<a href="https://colab.research.google.com/github/laura-katiuscia/2-Projeto/blob/master/Machine_Learning_com_Analise_de_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Entendo conceito de Pipeline com Python para Machine Learning (e um conceito que

# pode ser usada nas ling. Java, Python etc..Ela começa antes DATALAKE E DATAHOUSE).

# 📌 Pipeline simples (funções em sequência)
def limpar_dados(dados):
    return [x for x in dados if x is not None]

def normalizar(dados):
    max_val = max(dados)
    return [x / max_val for x in dados]

def treinar_modelo(dados):
    print("Treinando modelo com:", dados)

# Pipeline
dados = [10, 20, None, 30]
dados = limpar_dados(dados)
dados = normalizar(dados)
treinar_modelo(dados)

#📌 Ideia-chave: saída de uma função vira entrada da próxima.



In [ ]:
# Carregar arquivo colab
from google.colab import files
uploaded = files.upload()


In [ ]:
# lê o arquivo já carregado ✔️ Muito usado quando você quer limitar leituras de arquivos grandes.
from itertools import islice

with open('producao_alimentos.csv', 'r') as f:
    for linha in islice(f, 6):
        print(linha, end='')



In [ ]:
#Essa parte toda mas segunda parte do código serve para testar a conectividade
import csv
import sqlite3

#Cria novo banco de dados
conn = sqlite3.connect('dsadb.db')

# Adiciona esta linha para dropar a tabela se ela já existir
conn.execute('DROP TABLE IF EXISTS producao')

#Cria tabela para armazenar dos dados do arquivo producao_alimentos.csv
conn.execute('''CREATE TABLE producao (
            produto TEXT,
            quantidade INTEGER,
            preco_media REAL,
            receita_media REAL
)''')
conn.commit()
conn.close()

In [ ]:
#abre o arquivo CSV com os dados de produção de alimentos
with open('producao_alimentos.csv', 'r') as file:

#Cria um leitor de CSV para ler o arquivo
  reader = csv.reader(file)

  # pula primeira linha que contém os cabeçalhos das colunas
  next (reader)

  #Conecta ao banco de dados
  conn = sqlite3.connect('dsadb.db')


  for row in reader:

      conn.execute('INSERT INTO producao (produto, quantidade, preco_media, receita_media) VALUES (?, ?, ?, ?)', row)


  conn.commit()
  conn.close()

  print("Job Concluído com Sucesso!")

In [ ]:
#Esse código foi feito para responder o que foi pedido carrega somente quantidade
#prouzida maior que 10 da coluna quantidade_produzida_kls  OBS: Essa atividade
#JOB_V3.PY QUE E TRANSFORMAÇÃO DOS DADOS.
import csv
import sqlite3

# Função para remover o ponto nos dados da última coluna
def remove_ponto(valor):
    return int(valor.replace('.', '')) #Essa função usa replace que indetifica
    #ponto e substitui por virgula e no final converte para número inteiro.

# Abre o arquivo CSV com os dados da produção de alimentos
with open('producao_alimentos.csv', 'r') as file:

    # Cria um leitor de CSV para ler o arquivo
    reader = csv.reader(file)

    # Pula a primeira linha, que contém os cabeçalhos das colunas
    next(reader)

    # Conecta ao banco de dados
    conn = sqlite3.connect('dsadb.db')

    # Deleta a tabela existente, se houver
    conn.execute('DROP TABLE IF EXISTS producao')  #Aula Prática 04, TRANSFORMAÇÃO
    # DE DADOS, tirou os pontos que não aparecia a casa decimal dos zeros dos valores.

    # Cria uma nova tabela para armazenar os dados de produção de alimentos
    conn.execute('''CREATE TABLE producao (
                    produto TEXT,
                    quantidade INTEGER,
                    preco_medio REAL,
                    receita_total INTEGER,
                    margem_lucro REAL
                )''')

    #AQUI NO EXECUTE E DIFERENÇA DO EXERCICIO 03 PARA 04 ONDE SÓ ACRESCENTA coluna
    # com nome margem_lucro com resultado das colunas existentes que fez calculo

    # Insere cada linha do arquivo com quantidade maior que 10 na tabela do banco de dados
    #Aqui mantên a regra de negócio
    for row in reader:
        if int(row[1]) > 10:

            # Remove o ponto do valor da última coluna e converte para inteiro
            row[3] = remove_ponto(row[3]) #ùltima linha que e receita_total e
            #chama função criada com nome remove_ponto

            margem_lucro = (row[3] / float(row[1])) - float (row[2]) # Calculo 1º subtrai linha 1 menos a linha 2
            #row.append(margem_lucro)

            # Insere o registro no banco de dados sendo que margem de lucro não existe na origem isso deixa ele pronto pra uso
            #coluna margem e pra colocar resultado das colunas já existentes
            conn.execute('INSERT INTO producao (produto, quantidade, preco_medio, receita_total, margem_lucro) VALUES (?, ?, ?, ?, ?)', (row[0], row[1], row[2], row[3], margem_lucro))

    conn.commit()
    conn.close()

print("Job Concluído com Sucesso! Para vê o resultado abra no ")

In [ ]:
# REGRA DE NEGÓCIO Enriquecer os dados adicionando no destino uma coluna com margem
# de lucro de cadas produto

In [ ]:
# 📌 Pipeline com Pandas (Análise de Dados), muito usado no pré-processamento de dados %%

import pandas as pd

df = pd.DataFrame({
    "idade": [20, 25, None, 30],
    "salario": [2000, None, 3000, 4000]
})

pipeline = (
    df
    .dropna()
    .assign(
        idade_norm=lambda x: x["idade"] / x["idade"].max(),
        salario_norm=lambda x: x["salario"] / x["salario"].max()
    )
)

print(pipeline)

#📌 Ideal para ETL (Extract, Transform, Load).


In [ ]:
# 🚀 Pipeline profissional com scikit-learn (Machine Learning)

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris

# Dados
X, y = load_iris(return_X_y=True)

X_treino, X_teste, y_treino, y_teste = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Pipeline
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("modelo", LogisticRegression(max_iter=200))
])

# Treinar
pipeline.fit(X_treino, y_treino)

# Avaliar
print("Acurácia:", pipeline.score(X_teste, y_teste))

#🔥 O que acontece aqui? StandardScaler → normaliza os dados, LogisticRegression → treina o modelo, Tudo acontece automaticamente em ordem

# 📌 Evita erros, vazamento de dados e facilita deploy.


In [ ]:
# 4️⃣ Pipeline com colunas diferentes (numéricas e categóricas), Muito comum em projetos reais.

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

preprocessador = ColumnTransformer([
    ("num", StandardScaler(), ["idade", "salario"]),
    ("cat", OneHotEncoder(), ["sexo"])
])

pipeline = Pipeline([
    ("prep", preprocessador),
    ("modelo", RandomForestClassifier())
])

#Esse pipeline NÃO gera resposta sozinho. A resposta do modelo só aparece depois que você treina e faz uma predição.

# 👉 A resposta final do pipeline será: Uma classe prevista, por exemplo: 0 → não compra, 1 → compra


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Criando um dataset dummy para treino (com idade, salario, sexo e um target)
# Isso é necessário porque o pipeline com ColumnTransformer espera dados com essas colunas para ser treinado
data_treino = pd.DataFrame({
    "idade": [25, 35, 45, 22, 38, 50],
    "salario": [2500, 3500, 4500, 2200, 3800, 5000],
    "sexo": ["F", "M", "F", "M", "F", "M"],
    "target": [0, 1, 0, 1, 0, 1]
})

# Separando features (X) e target (y) para o treino
X_treino_dummy = data_treino[["idade", "salario", "sexo"]]
y_treino_dummy = data_treino["target"]

# Treinar o pipeline com os dados dummy
pipeline.fit(X_treino_dummy, y_treino_dummy)

novo_cliente = pd.DataFrame({
    "idade": [30],
    "salario": [4000],
    "sexo": ["F"]
})

resposta = pipeline.predict(novo_cliente)
probabilidade = pipeline.predict_proba(novo_cliente)

print("Classe prevista:", resposta)
print("Probabilidade:", probabilidade)

#Resposta classe prevista 0 sig. que ele não vai comprar

In [ ]:
#Tipos de dados
# LISTAS, DICIONARIOS E TUPLAS

Tupla = (1, 2, 3, 5, 6 , 8, 8)
Tupla
type(Tupla)
Tupla.count(8)

In [ ]:
def cor_fruta(cor):
    if (cor == 'laranja'):
        return 'É uma laranja'
    elif(cor == 'amarelo'):
        return 'É uma banana'
    elif(cor == 'verde'):
        return 'É uma manga'
    elif(cor =='preto'):
        return "amora"
    else:
        return 'Não conheço essa cor ou fruta! '

In [ ]:
fruta = cor_fruta('preto')
print(fruta)

In [ ]:
fruta = cor_fruta('azul')
print(fruta)

In [ ]:
dicio = {'Index' : 'valor',
'Id' : 1,
'Id' : 2,
'Id' : 3,
'Nome' : 'Sandro'
}
dicio

In [ ]:
lista = [1, 'Laura',[1, 2, 3, 4, 5, 6]]
lista

In [ ]:
exemplo = 3.14159
round(exemplo, 3)

In [ ]:
minha = 'Estudar Fatiamento e TOP'
print (len(minha))

In [ ]:
minha[1]


In [ ]:
minha [8]

In [ ]:
Nome = 'Laura katiuscia'
Nome.upper()


In [ ]:
Nome = 'Laura katiuscia'
Nome.lower()

In [ ]:
Frase = 'Estudar Fatiamento e TOP'
Frase.split()

In [ ]:
Frase = 'Estudar Fatiamento e TOP'
Frase.find('n')

In [ ]:
#strip Ele tira os espaços antes e depois, mas não altera o meio.
Frase = '  Estudar Fatiamento e TOP'
Frase.strip()

In [ ]:
nome = input('Qual é seu nome ?')
idade = input('Qual é sua idade ?')
print('Oi', nome, 'você tem',  idade , 'anos')

In [ ]:
'CPF' in Frase

In [ ]:
'demais' not in Frase

In [ ]:
Lista_vazia = []
Lista_vazia.append(1)
Lista_vazia.append(2)
Lista_vazia.append(3)
Lista_vazia.append(4)
Lista_vazia

In [ ]:
Lista_vazia[2]

In [ ]:
len(Lista_vazia)

In [ ]:
Lista_vazia.insert(1,2)
print(Lista_vazia)

In [ ]:
#coloca na índice 3 o número 7 e empurra o 3 e 4 para depois dele Lista_vazia.insert(3,7)
print(Lista_vazia)

In [ ]:
import datetime

nova_data = datetime.datetime.today()
print(nova_data)

In [ ]:
type(nova_data)

In [ ]:
print(nova_data.day)
print(nova_data.month)
print(nova_data.year)

In [ ]:
print(datetime.datetime.today().date())


In [ ]:
Dia = nova_data.day
print(Dia)

In [ ]:
nova_data.strftime('%d/%m/%Y')

In [ ]:
import time
print(time.localtime())
time.sleep(5)
print('Dormiu por 5 segundos')

In [ ]:
for conte in range(10):
  if conte % 2 == 0:
    print(conte)

In [ ]:
Parar = 0

while Parar <=10:

  print( Parar )
  Parar += 1


In [ ]:
Lista = ['Morango', 'Pera', 'Uva', 'Abacaxi', 'Quiui']
for fruta in Lista:
  print(fruta)
  if fruta == 'Uva':
    break

In [ ]:
Lista = ['Morango', 'Pera', 'Uva', 'Abacaxi', 'Quiui']
for fruta in Lista:
  print(fruta)
  if fruta == 'Abacaxi':
    break

In [ ]:
def matematica(x, y):
  return x + y




In [ ]:
matematica(10,40)

In [ ]:
def matematica(x, y):
  soma = x + y
  subtracao = x - y
  multiplicacao = x * y
  divisao = x / y
  return soma, subtracao, multiplicacao, divisao

In [ ]:
matematica(x=10,y=40)

In [ ]:
print (matematica(x=12, y=30))

In [ ]:


pd.options.display.float_format = '{:.2f}'.format # Números aparece 2 casas decimais, dados carregados direto link não fica salvo qd for fazer aulas novamento vai ter que carregar porém se quer permanente salvar google diver
import sys
print(sys.version)




Let's check the validity of the Excel file link using the `requests` library. This will send an HTTP GET request to the URL and print the status code, which can help us understand if the file is accessible.

If the status code is still `403 Forbidden` or another error, it confirms that the direct download link is not working. In that case, you might need to:

1. **Check for an alternative, public link** on the source website.
2. **Download the file manually** from its source and then upload it directly to your Colab environment using `files.upload()` or by mounting your Google Drive.
3. **Contact the data provider** to inquire about access permissions or an updated link.

In [ ]:
#Tratamento de erro
try:
    # A linha abaixo (1 / 0) irá gerar um erro de divisão por zero.
    # Se essa linha for comentada, "Deu certo" será impresso.
    # Se for descomentada, "Deu errado" será impresso.
    4 / 2
    print("Deu certo")
except: # Captura o erro específico de divisão por zero
    print("Deu errado")
# A linha 'True' do código original não tinha um propósito aparente para o erro.
# A linha '#0/0' era um comentário e não causava erro.

finally:
    print("Finally deu certo ou deu errado vai ser executado da mesma forma! ")

In [ ]:
#O construtor em Python é o método __init__. Def é uma função que pode ter outro nomes

#Ele roda automaticamente ao criar um objeto.

#Usado para definir atributos iniciais.

# parâmetro self para acessar valor que está dentro do método construtor

class Pessoa:
  def __init__(self, nome, idade): #Def --init-- e método construdor os valores serão passados pra ele.
    self.nome = nome
    self.idade = idade

  def Boas_vidas(self):
    print('Boas vinda(o)', self.nome)

  def Recusado(self):
    print('Seu acesso foi recusado!')

  def Maior_idade(self):
    if self.idade >= 18:
      print('Maior de idade !')
      self.Boas_vidas()
    else:
      print('Menor de idade !')
      self.Recusado()

In [ ]:
p1 = Pessoa('Marcela', 89)

In [ ]:
p1.Maior_idade()

In [ ]:
#Chamei novamente objeto p1 referenciei função Maior_idade onde ele executa  a lógica que tá dentro da função e imprime a resposta

